In [ ]:

import pygame
import math
import random
from pygame import Rect

WIDTH, HEIGHT = 960, 480
FPS = 60
TILE = 48

GRAVITY = 0 # float in water
MAX_FALL = 14
PLAYER_SPEED = 3
JUMP_VEL = 9

WHITE = (240, 240, 240)
BG = (20, 24, 28)

LEVELS = [
    [
        "....................",
        "#..................O",
        "#..................O",
        "#..................O",
        "#..................O",
        "O..................O",
        "O..................O",
        "O..................O",
        "O...S........P......",
        "######O#############",
    ],
    [
        "####################",
        "#..................#",
        "#..^...............#",
        "#...........#####..#",
        "#..................#",
        "#......P...........#",
        "#..........^.......#",
        "#..............C...#",
        "#..................#",
        "####################",
    ],
    [ 
        "####################",
        "##TT##vv##v#v###vvv#",
        "##TTTATTTvTTTvTTTTTT",
        "#BT^##TTTTT*TTTT***#", 
        "##TT##**T*T#T*T*####", 
        "#BT^####T#T#T#T#####", 
        "##TT################",
        "#BT^################", 
        "##PT################", 
        "##TT################"
    ]
]

LEVEL_BGS = [
    "C:/Users/ADMIN/Downloads/pygame_assets/level3bg.png",
    "C:/Users/ADMIN/Downloads/pygame_assets/level3bg.png",
    "C:/Users/ADMIN/Downloads/pygame_assets/level3bg.png"
]

def load_frames(sheet_path, frame_w, frame_h, num_frames):
    sheet = pygame.image.load(sheet_path).convert_alpha()
    frames = []
    for i in range(num_frames):
        frame = pygame.Surface((frame_w, frame_h), pygame.SRCALPHA)
        frame.blit(sheet, (0, 0), pygame.Rect(i*frame_w, 0, frame_w, frame_h))
        frames.append(frame)
    return frames

class Player:
    def __init__(self, x, y):
        self.rect = Rect(x - 8, y, 30, 30)
        self.pos = pygame.Vector2(x, y)
        self.vel_x = 0
        self.vel_y = 0
        self.health = 1
        self.dead = False
        self.won = False
        self.facing_right = True
        self.in_water = True
        self.gravity_effect_active = False
        self.animations = {
            "swim": load_frames("C:/Users/ADMIN/Downloads/pygame_assets/swimming.png", 48, 48, 3),
            "float":  load_frames("C:/Users/ADMIN/Downloads/pygame_assets/idle_underwater.png", 48, 48, 4),
            "die": load_frames("C:/Users/ADMIN/Downloads/pygame_assets/dying-effect-sheet.png", 48, 48, 7)
        }
        self.state = "float"
        self.frame_index = 0
        self.frame_speed = 0.15
        self.image = self.animations[self.state][0]
        self.mask = pygame.mask.from_surface(self.image)

    def animate(self):
        frames = self.animations[self.state]
        self.frame_index += self.frame_speed
        if self.frame_index >= len(frames):
            self.frame_index = 0
        self.image = frames[int(self.frame_index)]
        self.mask = pygame.mask.from_surface(self.image)

    def update(self):
        pass
    
    def set_state(self, new_state):
        if new_state != self.state:
            self.state = new_state
            self.frame_index = 0
            self.image = self.animations[self.state][0]
            self.mask = pygame.mask.from_surface(self.image)

    def handle_input(self, keys):
        if self.health <= 0 or self.won:
            self.vel_x = 0
            self.vel_y = 0
            return
        ax = 0
        if keys[pygame.K_a] or keys[pygame.K_LEFT]:
            ax -= 1
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
            ax += 1
        self.vel_x = ax * (PLAYER_SPEED * 0.7)
        if keys[pygame.K_w] or keys[pygame.K_UP]:
            self.vel_y = -JUMP_VEL * 0.1
        elif keys[pygame.K_s] or keys[pygame.K_DOWN]:
            self.vel_y = JUMP_VEL * 0.1
        else:
            self.vel_y *= 0.96  # chỉnh độ chìm theo trục y 
        if self.vel_x > 0:
            self.facing_right = True
        elif self.vel_x < 0:
            self.facing_right = False

    def move_and_collide(self, solids):
        if self.gravity_effect_active:
            # gravity nhẹ → chìm từ từ
            applied_gravity = 0.03
        else:
            applied_gravity = GRAVITY 
        self.vel_y += applied_gravity # thay đổi tốc độ di chuyển theo trục y, bị chìm xuống
        self.vel_y = max(min(self.vel_y, 1), -MAX_FALL) # giới hạn tốc độ
        self.rect.x += self.vel_x
        for s in solids:
            if self.rect.colliderect(s.rect):
                if self.vel_x > 0:
                    self.rect.right = s.rect.left
                elif self.vel_x < 0:
                    self.rect.left = s.rect.right
        self.rect.y += self.vel_y
        for s in solids:
            if self.rect.colliderect(s.rect):
                if self.vel_y > 0:
                    self.rect.bottom = s.rect.top
                elif self.vel_y < 0:
                    self.rect.top = s.rect.bottom
                self.vel_y = 0
        self.pos.x = self.rect.centerx
        self.pos.y = self.rect.centery

    def update_state(self):
        if self.health <= 0:
            self.set_state("die")
            return
        if abs(self.vel_x) > 0.1 or abs(self.vel_y) > 0.1:
            self.set_state("swim")
            self.frame_speed = 0.16
        else:
            self.set_state("float")
            self.frame_speed = 0.10

    def update_animation(self):
        self.update_state()
        frames = self.animations[self.state]
        self.frame_index += self.frame_speed
        if self.frame_index >= len(frames):
            self.frame_index = 0
        self.image = frames[int(self.frame_index)]
        if not self.facing_right:
            self.image = pygame.transform.flip(self.image, True, False)
        self.mask = pygame.mask.from_surface(self.image)

    def update_death_animation(self):
        if self.health <= 0:
            self.state = "die"
            frames = self.animations[self.state]
            self.frame_index += 0.15
            if self.frame_index >= len(frames):
                self.frame_index = len(frames) - 1
            self.image = frames[int(self.frame_index)]
            if not self.facing_right:
                self.image = pygame.transform.flip(self.image, True, False)
            self.mask = pygame.mask.from_surface(self.image)
            self.dead = True

    def draw(self, surface):
        surface.blit(self.image, (self.rect.x - 8, self.rect.y))

class Water_Block:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 48, 48)
        self.image = pygame.image.load("C:/Users/ADMIN/Downloads/pygame_assets/water_surface.png").convert_alpha()
    def draw(self, surface):
        surface.blit(self.image, (self.rect.x, self.rect.y))

class Switch:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 48, 48)
        self.image_off = pygame.image.load("C:/Users/ADMIN/Downloads/pygame_assets/unactivated_switch.png").convert_alpha() 
        self.image_on = pygame.image.load("C:/Users/ADMIN/Downloads/pygame_assets/activated_switch.png").convert_alpha() 
        self.image = self.image_off
        self.mask = pygame.mask.from_surface(self.image)
        self.activated = False 

    def update(self, player: "Player", world: "World"): 
        if self.activated:
            self.image = self.image_on
            self.mask = pygame.mask.from_surface(self.image)
            return

        offset = (player.rect.x - self.rect.x, player.rect.y - self.rect.y)
        switch_mask = pygame.mask.from_surface(self.image)
        if switch_mask.overlap(player.mask, offset):
            self.activated = True         
            self.image = self.image_on
            self.mask = pygame.mask.from_surface(self.image)
            player.gravity_effect_active = True

    def draw(self, surface):
        surface.blit(self.image, (self.rect.x, self.rect.y))

class Block:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 48, 48)
        self.image = pygame.image.load("C:/Users/ADMIN/Downloads/pygame_assets/block.png").convert_alpha()
        self.mask = pygame.mask.from_surface(self.image)
    def draw(self, surface):
        surface.blit(self.image, (self.rect.x, self.rect.y))

class BubbleSpout:
    def __init__(self, x, y): 
        self.frame_index = 0
        self.frames = load_frames("C:/Users/ADMIN/Downloads/fish_bubble.png",96,48,6)
        self.image = self.frames[self.frame_index]
        self.rect = pygame.Rect(x,y,96,27)
        self.anim_speed = 0.035
        self.anim_timer = 0
        self.spray_frames = {2, 3} 
        self.is_spraying = False
        self.mask = pygame.mask.from_surface(self.image)

    def update(self):
        self.anim_timer += self.anim_speed
        if self.anim_timer >= 1:
            self.anim_timer = 0
            self.frame_index = (self.frame_index + 1) % len(self.frames)
            self.image = self.frames[self.frame_index]
            self.mask = pygame.mask.from_surface(self.image) # dùng mask là cái lấy khung cho image theo pixel, tạo va chuẩn, smooth
        self.is_spraying = self.frame_index in self.spray_frames

    def draw(self, screen):
        screen.blit(self.image, self.rect)
        
    def check_collision_and_knockback(self, player):
        if self.is_spraying:
            offset = (player.rect.x - self.rect.x, player.rect.y - self.rect.y)
            if self.mask.overlap(player.mask, offset): # va chạm thì sẽ đẩy player qua phải
                player.rect.x += 5

# đoạn này t thấy k cần tách ra thành 2 layers vì con cá cũng để lùi dần và cái trap này cũng k gây die 

class Spike:
    def __init__(self, x, y, direction='left'): 
        self.rect = pygame.Rect(x, y, 48,30)
        self.frames = load_frames("C:/Users/ADMIN/Downloads/pygame_assets/trap-1-sheet.png", 48, 48, 2)
        self.frame_index = 0
        self.frame_speed = 0
        self.direction = direction
        self.image = self.frames[0]
        self.mask = pygame.mask.from_surface(self.image)

    def update(self, player):
        if self.frame_speed > 0:
            self.frame_index += self.frame_speed
            if self.frame_index >= len(self.frames):
                self.frame_index = 0
            self.image = self.frames[int(self.frame_index)]
            self.mask = pygame.mask.from_surface(self.image)
        offset = (player.rect.x - self.rect.x, player.rect.y - self.rect.y)
        if self.mask.overlap(player.mask, offset):
            player.health = 0

    def draw(self, surface):
        current_frame = self.frames[int(self.frame_index)]
        if self.direction == 'down':
            current_frame = pygame.transform.rotate(current_frame, 90) 
        elif self.direction == "up":
            current_frame = pygame.transform.rotate(current_frame, -90) 
        surface.blit(current_frame, (self.rect.x, self.rect.y))

class World:
    def __init__(self, ascii_map, bg_path=None):
        self.blocks = []
        self.water_blocks = []
        self.spikes = []
        self.water_spouts = []
        self.switches = []
        self.player_start = (64, 64)
        self.bubble_spouts = []

        if bg_path:
            self.bg_image = pygame.image.load(bg_path).convert()
        else:
            self.bg_image = None

        full_cols = WIDTH // TILE
        full_rows = HEIGHT // TILE
        for r in range(full_rows):
            for c in range(full_cols):
                self.water_blocks.append(Water_Block(c * TILE, r * TILE))

        for j, row in enumerate(ascii_map):
            for i, cell in enumerate(row):
                x, y = i * TILE, j * TILE
                layers = list(cell) if isinstance(cell, str) else [cell]
                for ch in layers:
                    if ch == '#':
                        self.blocks.append(Block(x, y))
                    elif ch == '^':
                        self.spikes.append(Spike(x, y, direction='left')) 
                    elif ch == '*':
                        self.spikes.append(Spike(x, y, direction='up')) 
                    elif ch == 'v':
                        self.spikes.append(Spike(x, y, direction='down'))
                    elif ch == 'P':
                        self.player_start = (x + TILE // 2, y + TILE) 
                    elif ch == 'A':
                        self.switches.append(Switch(x, y))
                    elif ch == 'B':
                        self.bubble_spouts.append(BubbleSpout(x,y))
        self.player = Player(*self.player_start)

    def solids(self):
        return self.blocks 
    
    def update(self):
        for spout in self.bubble_spouts:
            spout.update()
            spout.check_collision_and_knockback(self.player)
        if self.player.health > 0 and not self.player.won:
            keys = pygame.key.get_pressed()
            self.player.handle_input(keys)
            self.player.move_and_collide(self.solids())
            self.player.update_animation()
        for sp in self.spikes:
            sp.update(self.player)
        for sw in self.switches:
            sw.update(self.player, self)
        if self.player.rect.right >= WIDTH:
            self.player.won = True

    def draw_background(self, surface):
        if self.bg_image:
            surface.blit(self.bg_image, (0, 0))

    def draw(self, surface):
        for wb in self.water_blocks:
            wb.draw(surface)
        for b in self.blocks:
            b.draw(surface)
        for sp in self.spikes:
            sp.draw(surface)
        for sw in self.switches:
            sw.draw(surface)
        for spout in self.bubble_spouts:
            spout.draw(surface)
        self.player.draw(surface)
        if self.player.won:
            font = pygame.font.Font("D:/Python NEU/PixelifySans-VariableFont_wght.ttf", 100)
            text = font.render("YOU WIN!!", True, 'gold')
            surface.blit(text, (WIDTH//2 - text.get_width()//2 + 60, HEIGHT//4 ))
        elif self.player.dead:
            font = pygame.font.Font("D:/Python NEU/PixelifySans-VariableFont_wght.ttf", 35)
            text = font.render("GAME OVER!! Press R to Restart", True, 'black')
            surface.blit(text, (WIDTH//2 - text.get_width()//2 + 70, HEIGHT//5))
            text1 = font.render("Fighting ^^ Trust me bro", True, 'red')
            surface.blit(text1,(WIDTH//2 - text.get_width()//2 + 130, HEIGHT //3.5))
        
def run():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Penguin Swimming Hero !")
    clock = pygame.time.Clock()

    current_level_index = 2 
    world = World(LEVELS[current_level_index], bg_path=LEVEL_BGS[current_level_index])
    running = True

    def restart(current_level_index):
        return World(
            LEVELS[current_level_index],
            bg_path=LEVEL_BGS[current_level_index]
        )

    while running:
        clock.tick(FPS)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        keys = pygame.key.get_pressed()
        player = world.player
        
        world.update() 

        if player.won and keys[pygame.K_RETURN]:
            running = False
            
        if player.health <= 0:
            player.update_death_animation()
            
        if player.dead and keys[pygame.K_r]:
            world = restart(current_level_index)
            player = world.player 
            player.dead = False
            
        world.draw_background(screen)
        world.draw(screen)
        
        pygame.display.flip()


if __name__ == "__main__":
    run()